<a href="https://colab.research.google.com/github/J-o-y-y/codeslave/blob/main/%EC%8A%A4%ED%8C%B8%ED%95%84%ED%84%B0%EB%A7%81_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 스팸필터링을 NBC(나이브베이즈분류 모델)을 쌩으로 구현해서 함수화.

1) 전처리 : 단어를 공백으로 분리하여, 단어별 빈도수 세기

2) 스팸/정상일 경우 조건부 확률 계산 : 각 단어빈도를 활용하여 스팸/정상일 경우 조건부 확률 계산 (라플라스 스무딩 적용)

3) 조건부 확률에 로그 적용

4) 스팸/정상에 대한 사전확률 계산 및 로그 적용

5) 베이즈 정리를 적용하여 **“Free Lottery”** 단어 등장시 스팸일 확률과 정상일 확률 계산

이를 함수로 구현.



training_set = [[‘me free lottery’, 1],
 [‘free get free you’, 1],
 [‘you free scholarship’, 0],
 [‘free to contact me’, 0],
 [‘you won award’, 0],
 [‘you ticket lottery’, 1]]
3:40
test = [‘free lottery’]

In [1]:
import math

# 훈련 데이터셋
training_set = [
    ['mefreelottery', '스팸메일'],
    ['freegetfreeyou', '스팸메일'],
    ['youfreescholarship', '정상메일'],
    ['freetocontactme', '정상메일'],
    ['youwonaward', '정상메일'],
    ['youticketlottery', '스팸메일']
]

# 전체 이메일 수
total_emails = len(training_set)

# 스팸과 정상 메일의 개수 계산
spam_count = sum(1 for email in training_set if email[1] == '스팸메일')
normal_count = sum(1 for email in training_set if email[1] == '정상메일')

# 사전확률 계산
p_spam = spam_count / total_emails
p_normal = normal_count / total_emails

# 모든 단어들을 공백을 기준으로 분리하고 빈도수 계산
word_counts = {}
for email, label in training_set:
    words = email.split()
    for word in words:
        if word in word_counts:
            word_counts[word][label] = word_counts[word].get(label, 0) + 1
        else:
            word_counts[word] = {label: 1}

# 라플라스 스무딩 적용 및 로그 적용
alpha = 1  # 라플라스 스무딩 하이퍼파라미터
vocabulary_size = len(word_counts)

def calculate_conditional_probability(word, label):
    count_word_label = word_counts.get(word, {}).get(label, 0)
    count_label = spam_count if label == '스팸메일' else normal_count
    return math.log((count_word_label + alpha) / (count_label + alpha * vocabulary_size))

# 입력 텍스트 분류
def classify_email(input_text):
    words = input_text.split()
    p_spam_given_words = math.log(p_spam)
    p_normal_given_words = math.log(p_normal)

    for word in words:
        p_spam_given_words += calculate_conditional_probability(word, '스팸메일')
        p_normal_given_words += calculate_conditional_probability(word, '정상메일')

    if p_spam_given_words > p_normal_given_words:
        return '스팸메일'
    else:
        return '정상메일'

# 테스트 데이터
test_email = 'Free Lottery'
result = classify_email(test_email)
print(f'입력 텍스트 "{test_email}"는 {result} 입니다.')


입력 텍스트 "Free Lottery"는 정상메일 입니다.
